## Сортировка дисциплин

### Версия Python

In [1]:
!python -V

Python 3.12.6


### Импорт необходимых библиотек

In [19]:
# Подавление предупреждений
import warnings
for warn in [UserWarning, FutureWarning]: warnings.filterwarnings("ignore", category = warn)

import os
import re
import jupyterlab as jlab
import ipywidgets
import pandas as pd
import tqdm as tq
import shutil
import glob

from pathlib import Path
from tqdm import tqdm

### Версии необходимых библиотек

In [20]:
packages = [
    "Pandas", "Ipywidgets", "JupyterLab", "Tqdm"
]

package_objects = [
    pd, ipywidgets, jlab, tq
]

versions = list(map(lambda obj: obj.__version__, package_objects))

columns_order = ["№", "Библиотека", "Версия"]
df_pkgs = (
    pl.DataFrame({
        columns_order[1]: packages,
        columns_order[2]: versions
    })
    .with_columns(pl.arange(1, pl.lit(len(packages)) + 1).alias(columns_order[0]))
    .select(columns_order)
)

display(df_pkgs)

path_to_reqs = "."
reqs_name = "requirements.txt"

def get_packages_and_versions():
    """Генерация строк с библиотеками и их версиями в формате: библиотека==версия"""
    
    for package, version in zip(packages, versions):
        yield f"{package.lower()}=={version}\n"

with open(os.path.join(path_to_reqs, reqs_name), "w", encoding = "utf-8") as f:
    f.writelines(get_packages_and_versions())

№,Библиотека,Версия
i64,str,str
1,"""Pandas""","""2.2.3"""
2,"""Ipywidgets""","""8.1.5"""
3,"""JupyterLab""","""4.2.5"""
4,"""Tqdm""","""4.66.5"""


### Чтение ПУДов

In [21]:
path_to_files = "/Users/dl/GitHub/ArenaVSRS/data/subjects"

xlsx_files = glob.glob(os.path.join(path_to_files, "*.xlsx"))

df_x = pd.DataFrame()

for file in xlsx_files:
    df_temp = pd.read_excel(file)
    df_x = pd.concat([df_x, df_temp], ignore_index = True)

df_x_unique = df_x.drop_duplicates(subset = ["ID дисциплины БУП ППК (АСАВ)"])

In [22]:
len(df_x_unique)

36850

In [23]:
df_x_unique['Уровень обучения'].unique()

array(['-', 'Бакалавриат', 'Аспирантура', 'Магистратура', nan,
       'Специалитет', 'Бакалавриат, Специалитет',
       'Бакалавриат, Магистратура'], dtype=object)

### Сортировка

In [ ]:
# Путь к исходной и целевой директориям
path_to_files = Path("/Users/dl/GitHub/ArenaVSRS/data/vacancies_subjects_association/paraphrase-multilingual-MiniLM-L12-v2")
save_path = Path("/Users/dl/GitHub/ArenaVSRS/data/vacancies_subjects_association/paraphrase-multilingual-MiniLM-L12-v2_sort")

# Приоритеты для сортировки
priority = {
    "Бакалавриат": 0,
    "Специалитет": 1,
    "Бакалавриат, Магистратура": 2,
    "Магистратура": 3,
    "Аспирантура": 4,
    "-": 5,
    "nan": 6 # если встречается nan
}

# Функция для извлечения числовых значений из l[-1]
def extract_numbers(value):
    # Извлекаем все числа
    numbers = re.findall(r'(\d+)\s+\w+', value)
    # Если чисел нет, возвращаем специальный флаг, например, None
    return [int(num) for num in numbers] if numbers else None

# Функция для проверки перехода через год
def is_year_transition(first_number, second_number):
    return first_number > second_number

# Кастомная функция сортировки
def custom_sort_key(r):
    l = r.strip().split("|")
    category = l[-2].strip() if len(l) > 1 else "nan"  # Категория (Бакалавриат и т.д.)
    
    numbers = extract_numbers(l[-1].strip())  # Извлеченные числа
    
    if numbers is None:
        # Если чисел нет, отправляем строку в конец текущей категории
        first_number, second_number, modules_sum = float('inf'), float('inf'), float('inf')
        weight = 2  # Устанавливаем вес, чтобы такие строки шли последними
    else:
        first_number = numbers[0]
        second_number = numbers[1] if len(numbers) > 1 else first_number
        modules_sum = abs(second_number - first_number)
        
        # Проверяем, происходит ли переход на следующий год
        weight = 1 if is_year_transition(first_number, second_number) else 0  # Переход на следующий год — приоритет ниже
    
    # Сортировка по приоритету, первому числу, весу (если есть переход) и количеству модулей
    return (priority.get(category, 7), first_number, weight, modules_sum, second_number)

# Проверка и создание/очистка целевой папки
if save_path.exists():
    shutil.rmtree(save_path)  # Удаляем все файлы и папки внутри
save_path.mkdir(parents=True, exist_ok=True)  # Создаем целевую папку

# Получение списка всех CSV файлов
csv_files = list(path_to_files.rglob("*.csv"))

# Проходим по каждому файлу
for file_path in tqdm(csv_files, desc="Обработка CSV файлов", unit="файл"):
    # Чтение CSV файла
    df_u = pd.read_csv(file_path, sep=';', encoding="utf-8-sig")

    # print(file_path)
    
    # Проходим по каждой строке файла
    for index, row in df_u.iterrows():
        # Сортировка на основе кастомного ключа
        sorted_row = sorted(row["SBERT_Disciplines"].split(";"), key=custom_sort_key)

        # Проверяем, есть ли данные в SBERT_plus_LLM_Recommendations
        recommendations_row = row["SBERT_plus_LLM_Recommendations"]
        
        # Если recommendations не пустой и не NaN, выполняем сортировку
        if isinstance(recommendations_row, str) and recommendations_row.strip() != "" and not pd.isna(recommendations_row):
            recommendations = recommendations_row.split(";")
            recommendations = [r.strip() for r in recommendations]

            # Извлекаем названия дисциплин из sorted_row
            discipline_names_in_sorted_row = [
                s.split("|")[1].strip() for s in sorted_row
            ]
        
            # Создаем словарь, сопоставляющий названия дисциплин с их индексом в sorted_row
            discipline_to_index = {name: i for i, name in enumerate(discipline_names_in_sorted_row)}
        
            # Функция для получения индекса дисциплины из словаря (или возвращения большого значения, если нет совпадения)
            def recommendation_sort_key(rec):
                return discipline_to_index.get(rec.strip(), float('inf'))
        
            # Сортируем список recommendations на основе порядка в sorted_row
            sorted_recommendations = sorted(recommendations, key=recommendation_sort_key)
        
            # Проверяем, какие дисциплины из recommendations отсутствуют в sorted_row
            missing_in_sorted_row = [rec for rec in recommendations if rec not in discipline_names_in_sorted_row]
        
            # Добавляем недостающие элементы в конец списка
            sorted_recommendations.extend(missing_in_sorted_row)
        
            # Результат в виде строки с разделителем "; "
            sorted_recommendations_str = "; ".join(sorted_recommendations)

            df_u.at[index, "SBERT_plus_LLM_Recommendations"] = sorted_recommendations_str

        # print(discipline_names_in_sorted_row)
        # print(sorted_recommendations)
        
        sorted_row = [s.strip() for s in sorted_row]

        sorted_row_copy = sorted_row.copy()

        for idx, s2 in enumerate(sorted_row_copy):  
            l = s2.strip().split("|")
            df_x_unique_1 = df_x_unique[df_x_unique["ID дисциплины БУП ППК (АСАВ)"] == int(l[0])]

            l.append(df_x_unique_1["Охват аудитории"].to_string(index=False))
            l.append(df_x_unique_1["Формат изучения"].to_string(index=False))

            l = " | ".join(list(map(str.strip, l)))

            sorted_row[idx] = l

        # print(sorted_row)
        sorted_row_s = "; ".join(sorted_row)

        # print(sorted_row_s)

        # Запись результата
        df_u.at[index, "SBERT_Disciplines"] = sorted_row_s

    # Определение пути для сохранения файла
    save_file_path = save_path / file_path.name
    
    # Сохранение обработанного DataFrame в CSV
    df_u.to_csv(save_file_path, index=False, encoding="utf-8-sig", sep=";")

Обработка CSV файлов:  22%|████████████▉                                             | 26/117 [21:10<41:28, 27.35s/файл]